In [47]:
import requests
from requests import request
from bs4 import BeautifulSoup as soup
from pprint import pprint
import re
import libtorrent
import time
import sys
import os.path
import difflib 
from collections import OrderedDict
import itertools
import urllib
from pprint import pprint


def get_latest_movirulz_url(keyword, n_results=1):
    headers = {"User-Agent": "Mozilla/5.0"}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    query = keyword
    query = urllib.parse.quote_plus(query)
    number_result = n_results
    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
    response = requests.get(google_url, headers=headers, cookies=cookies)
    soupp = soup(response.text, "html.parser")
    result = soupp.find_all('div')
    results=[re.search('\/url\?q\=(.*)\&sa',str(i.find('a', href = True)['href'])) for i in result if "url" in str(i)]
    links=[i.group(1) for i in results if i != None]
    return links[0]

movierulz_url = get_latest_movirulz_url('movierulz')+'?s='

movie = input("Enter Movie Name:")
movie = movie.replace(" ","%20")

url = movierulz_url+movie

response = request('GET',url)


Enter Movie Name:skylab


In [57]:
html_page_soup_1 = soup(response.text, "html.parser")
search_container = html_page_soup_1.findAll("div", {"class": "content home_style"})
try:
    movie_titles =  search_container[0].findAll("b")
except:
    print("could'nt find movie")
movie_title_ls = [] ; trimmed_movie_title_ls = []

rep = {"Full": "", "Movie": "","Watch":" ","Online":"","Free":""}
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))

for i in movie_titles:
    movie_title_ls.append(i.text.strip())
    trimmed_movie_name = pattern.sub(lambda m: rep[re.escape(m.group(0))],i.text.strip())
    trimmed_movie_title_ls.append(trimmed_movie_name.strip())
    
if len(trimmed_movie_title_ls)==1:
    best_match = trimmed_movie_title_ls[0]
else:
    best_match = difflib.get_close_matches(movie,trimmed_movie_title_ls,n=1,cutoff=0.2)[0]

close_results=dict()
for _ in trimmed_movie_title_ls:
    score = difflib.SequenceMatcher(None, _, best_match).ratio()
    close_results.update({score:_})
close_results = OrderedDict(sorted(close_results.items(),reverse=True))   
close_results = dict(itertools.islice(close_results.items(), 5))
#print(close_results)

close_results_ls = [*close_results.values()]
print("close_results_ls:", close_results_ls)

pprint(close_results_ls[0])

get_torrent(close_results_ls[0])

close_results_ls: ['Skylab (2021) HDRip Original [Telugu + Tamil + Malayalam + Kannada]']
'Skylab (2021) HDRip Original [Telugu + Tamil + Malayalam + Kannada]'
Skylab-(2021)-HDRip-Original-Telugu-+-Tamil-+-Malayalam-+-Kannada
[<small>2.1 gb</small>, <small>1080p</small>, <small>1.5 gb</small>, <small>1080p</small>, <small>1.2 gb</small>, <small>720p</small>, <small>850 mb</small>, <small>720p</small>, <small>700 mb</small>, <small>320p</small>, <small>450 mb</small>, <small>320p</small>, <small>250 mb</small>, <small>240p</small>] 

[('2.1 gb', '1080p'),
 ('1.5 gb', '1080p'),
 ('1.2 gb', '720p'),
 ('850 mb', '720p'),
 ('700 mb', '320p'),
 ('450 mb', '320p'),
 ('250 mb', '240p')]


In [56]:
def get_torrent(movie_name):
    
    movie = movie_name.replace("[","")
    movie = movie.replace("]","")
    movie = movie.replace(" ","-")
    
    base_url= "https://ww5.7movierulz.tc/"
    url = base_url+movie
    response = request('GET',url)
    print(movie)
    html_page_soup = soup(response.text, "html.parser")
    container = html_page_soup.findAll("a", {"class": "mv_button_css"})
    movie_title = html_page_soup.findAll("h2", {"class": "entry-title"})

    torrent_quality = html_page_soup.findAll("small")
    movie_title = movie_title[0].text.strip()
    
    print(torrent_quality,"\n")

    torrent_size_ls = [] ; torrent_resolution_ls = [] ; magnet_link_ls = [] ; magnets = dict()
    for i,torrent in enumerate(torrent_quality):
        if i%2==0:
            torrent_size_ls.append(torrent.text.strip())
        else:
            torrent_resolution_ls.append(torrent.text.strip())

    torrent_info = dict(zip(torrent_size_ls,torrent_resolution_ls))
    
    keyorder = ['8K','4K','4K SDR','1080p','1440p','720p', '480p', '320p','240p','144p']
    torrent_info = sorted(torrent_info.items(), key=lambda i:keyorder.index(i[1]))
    
    pprint(torrent_info)
    for link in container:
        magnet_link_ls.append(link.get('href'))

    for i,j in zip(magnet_link_ls,torrent_info):
        magnets.update({j[0]:i})

    #pprint(magnets)
    


In [ ]:
def mag2tor(name,mag_link):
    sess = libtorrent.session()
    prms = {
        'save_path':os.path.abspath(os.path.curdir),
    }
    torr = libtorrent.add_magnet_uri(sess, mag_link, prms)
    dots = 0
    while not torr.has_metadata():
        dots += 1
        sys.stdout.write('.')
        sys.stdout.flush()
    if (dots): sys.stdout.write('\n')
    sess.pause()
    tinf = torr.get_torrent_info()
    f = open(name + '.torrent', 'wb')
    f.write(libtorrent.bencode(
        libtorrent.create_torrent(tinf).generate()))
    f.close()